## Tarbimise ja tootmise andmete allalaadimine

- loeme CSV failist tarbimise ja tootmise andmed.
- salvestame andmed faili `data/raw_consumption.json` ja `data/raw_production.json`

In [ ]:
import pandas as pd
from pathlib import Path

In [15]:
project_root = Path("..").resolve()
data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

Laeme andmes .csv failist.

In [14]:
csv_path = Path("..") /"data"/"electricity_production_and_consumption_20251211.csv"

df = pd.read_csv(csv_path, sep=";", encoding="latin1")
df.head()

,Ajatempel (UTC),Kuupäev (Eesti aeg),Tarbimine,Tootmine,Planeeritud tarbimine,Planeeritud tootmine
0,1765404000,11.12.2025 00:00,"909,53","604,47","903,5","652,9"
1,1765404900,11.12.2025 00:15,"907,69","600,56","890,1","631,7"
2,1765405800,11.12.2025 00:30,"884,03","598,78","877,9",631
3,1765406700,11.12.2025 00:45,"871,78","602,21","863,9","629,7"
4,1765407600,11.12.2025 01:00,"846,06","649,58","830,6","642,8"


Puhastame veerud.

In [17]:
numeric_cols = ["Tarbimine", "Tootmine", "Planeeritud tarbimine", "Planeeritud tootmine"]

for col in numeric_cols:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

df.head()

,Ajatempel (UTC),Kuupäev (Eesti aeg),Tarbimine,Tootmine,Planeeritud tarbimine,Planeeritud tootmine
0,1765404000,11.12.2025 00:00,909.53,604.47,903.5,652.9
1,1765404900,11.12.2025 00:15,907.69,600.56,890.1,631.7
2,1765405800,11.12.2025 00:30,884.03,598.78,877.9,631.0
3,1765406700,11.12.2025 00:45,871.78,602.21,863.9,629.7
4,1765407600,11.12.2025 01:00,846.06,649.58,830.6,642.8


Lisame Timestampi

In [18]:
df["timestamp"] = pd.to_datetime(df["Ajatempel (UTC)"], unit="s")
df = df.sort_values("timestamp").reset_index(drop=True)

df[["Ajatempel (UTC)", "timestamp"]].head()

,Ajatempel (UTC),timestamp
0,1765404000,2025-12-10 22:00:00
1,1765404900,2025-12-10 22:15:00
2,1765405800,2025-12-10 22:30:00
3,1765406700,2025-12-10 22:45:00
4,1765407600,2025-12-10 23:00:00


Koostame tarbimise JSON faili.

In [19]:
df_consumption = df[["timestamp", "Tarbimine"]].copy()
df_consumption.rename(columns={"Tarbimine": "consumption_mwh"}, inplace=True)

df_consumption.to_json(data_dir / "raw_consumption.json", orient="records", date_format="iso")

df_consumption.head()


,timestamp,consumption_mwh
0,2025-12-10 22:00:00,909.53
1,2025-12-10 22:15:00,907.69
2,2025-12-10 22:30:00,884.03
3,2025-12-10 22:45:00,871.78
4,2025-12-10 23:00:00,846.06


Koostame tootmise JSON faili.

In [20]:
df_production = df[["timestamp", "Tootmine"]].copy()
df_production.rename(columns={"Tootmine": "production_mwh"}, inplace=True)

df_production.to_json(data_dir / "raw_production.json", orient="records", date_format="iso")

df_production.head()

,timestamp,production_mwh
0,2025-12-10 22:00:00,604.47
1,2025-12-10 22:15:00,600.56
2,2025-12-10 22:30:00,598.78
3,2025-12-10 22:45:00,602.21
4,2025-12-10 23:00:00,649.58


Koostame planeeritud tarbimise JSON faili.

In [24]:
df_forecast = df[["timestamp", "Planeeritud tarbimine"]].copy()
df_forecast.rename(columns={"Planeeritud tarbimine": "forecast_consumption_mwh"}, inplace=True)

forecast_path = data_dir / "raw_forecast_consumption.json"
df_forecast.to_json(forecast_path, orient="records", date_format="iso")

forecast_path, df_forecast.head()

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_consumption.json'),
             timestamp  forecast_consumption_mwh
 0 2025-12-10 22:00:00                     903.5
 1 2025-12-10 22:15:00                     890.1
 2 2025-12-10 22:30:00                     877.9
 3 2025-12-10 22:45:00                     863.9
 4 2025-12-10 23:00:00                     830.6)

Koostame planeeritud tootmise JSON faili.

In [25]:
df_forecast = df[["timestamp", "Planeeritud tootmine"]].copy()
df_forecast.rename(columns={"Planeeritud tootmine": "forecast_production_mwh"}, inplace=True)

forecast_path = data_dir / "raw_forecast_production.json"
df_forecast.to_json(forecast_path, orient="records", date_format="iso")

forecast_path, df_forecast.head()

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_production.json'),
             timestamp  forecast_production_mwh
 0 2025-12-10 22:00:00                    652.9
 1 2025-12-10 22:15:00                    631.7
 2 2025-12-10 22:30:00                    631.0
 3 2025-12-10 22:45:00                    629.7
 4 2025-12-10 23:00:00                    642.8)

- Importisin tarbimise ja tootmise andmed CSV failist.
- Normaliseerisin numbriveerud.
- Teisendasin timestampi UNIX sekunditest kuupäevaks.
- Lõin JSON failid:
    - `raw_consumption.json`
    - `raw_production.json`
    - `raw_forecast_consumption.json`
    - `raw_forecast_production.json`